In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('/Users/markwindsor/Desktop/atlas_trade')
from src.utils.technical_indicators import *
import warnings
warnings.filterwarnings("ignore")
plt.style.use("seaborn-v0_8")

class MeanRevertingCrypto():

    def __init__(self, data, symbol, trading_costs):
        self.symbol = symbol
        self.trading_costs = trading_costs
        self.results = None
        self.data_df = self.set_data(data)
        
    def __repr__(self):
        return "Mean Reverting Crypto (symbol = {})".format(self.symbol)
    
    def set_data(self, data):
        data['returns'] = np.log(data.close / data.close.shift(1))
        data.set_index('time', inplace=True)
        return data.dropna()  # Remove NaN values caused by shift
    
    def prepare_data(self):
        data = self.data_df.copy()
        data = relative_strength_index(data)
        data = bollinger_bands(data)
        data.dropna(inplace=True)
        
        # Buy and Sell Conditions
        data["position"] = np.where(
            (data['rsi'] < 30) & (data['close'] <= data['lower_band']), 1, 
            np.where((data['rsi'] > 70) & (data['close'] >= data['upper_band']), -1, 0)
        )
        self.results = data.copy()

    def run_backtest(self):
        if self.results is None:
            print("Run prepare_data() first.")
            return

        data = self.results.copy()
        data['strategy'] = data["position"].shift(1) * data["returns"]
        data["trades"] = data.position.diff().fillna(0).abs()
        data.strategy -= data.trades * self.trading_costs

        self.results = data

    def test_strategy(self):
        self.prepare_data()
        self.run_backtest()
        
        data = self.results.copy()
        data["creturns"] = data["returns"].cumsum().apply(np.exp)
        data["cstrategy"] = data["strategy"].cumsum().apply(np.exp)
        self.results = data
        self.print_performance()

    def plot_results(self):
        if self.results is None:
            print("Run test_strategy() first.")
            return

        title = "{} | Trading Costs = {}".format(self.symbol, self.trading_costs)
        self.results[["creturns", "cstrategy"]].plot(title=title, figsize=(12, 8))

    def print_performance(self):
        if self.results is None:
            print("No results to display.")
            return

        strategy_performance = round(self.calculate_multiple(self.results['strategy']), 6)
        print(100 * "=")
        print("Pure Technical| INSTRUMENT = {} |".format(self.symbol))
        print(100 * "-")
        print("PERFORMANCE MEASURES:")
        print("\n")
        print("Multiple (Strategy):         {}".format(strategy_performance))
        print(38 * "-")
        print(100 * "=")

    def calculate_multiple(self, series):
        return np.exp(series.sum())

data_path = '/Users/markwindsor/Desktop/atlas_trade/src/data/ETHUSDT_30days.csv'
data = pd.read_csv(data_path)
symbol = "ETHUSDT"
tc = -0.0005

tester = MeanRevertingCrypto(data, symbol, tc)
tester.test_strategy()